In [14]:
import spacy
from transformers import BertTokenizer, BertModel
from rdflib import Graph
import torch

In [16]:
nlp = spacy.load("en_core_web_sm")

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1716055056&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjA1NTA1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9iZXJ0LWJhc2UtdW5jYXNlZC82OGQ0NWUyMzRlYjRhOTI4MDc0ZGZkODY4Y2VhZDAyMTlhYjg1MzU0Y2M1M2QyMGU3NzI3NTNjNmJiOTE2OWQzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=MfP7%7EW284lnOTVZrvIkpwidJ6da6Y8avZKalzVAN4x53JPOAk0s3uMiGGQEFERCivqwa6KIDUlLcLgSBx7-pxThKQTkV40d9VppQiW1lOCipXs50rdiUhElWXeNfjqny4KfiKubSlAtcKJNnd%7Etfp8oON9rhiNZwESDnHjb0ibzixbgMLSdOribr5VtMvtg4At61hlltIU9N10ejOZzJAsK3DzJEBL4pf8YDn5EFy%7ExEvVYwv46aLezeobpks%7E6-KolxY4H4Fr7v1QsSvRdgCnZCatD8RIBXXywDBjRZMlZ-Wl6A7wlf1Go9lLDio5Rlnwl18IYSW-IgwIJxkC5Ovg__&Key-Pair-Id=KVTP0A1DKRTAX: HTT

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/bert-base-uncased/68d45e234eb4a928074dfd868cead0219ab85354cc53d20e772753c6bb9169d3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1716055056&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjA1NTA1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9iZXJ0LWJhc2UtdW5jYXNlZC82OGQ0NWUyMzRlYjRhOTI4MDc0ZGZkODY4Y2VhZDAyMTlhYjg1MzU0Y2M1M2QyMGU3NzI3NTNjNmJiOTE2OWQzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=MfP7%7EW284lnOTVZrvIkpwidJ6da6Y8avZKalzVAN4x53JPOAk0s3uMiGGQEFERCivqwa6KIDUlLcLgSBx7-pxThKQTkV40d9VppQiW1lOCipXs50rdiUhElWXeNfjqny4KfiKubSlAtcKJNnd%7Etfp8oON9rhiNZwESDnHjb0ibzixbgMLSdOribr5VtMvtg4At61hlltIU9N10ejOZzJAsK3DzJEBL4pf8YDn5EFy%7ExEvVYwv46aLezeobpks%7E6-KolxY4H4Fr7v1QsSvRdgCnZCatD8RIBXXywDBjRZMlZ-Wl6A7wlf1Go9lLDio5Rlnwl18IYSW-IgwIJxkC5Ovg__&Key-Pair-Id=KVTP0A1DKRTAX: HTT

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
g = Graph()
g.parse("knowledge_graph.rdf")

In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    expanded_text = " ".join([token.text for token in doc])
    return expanded_text

In [ ]:
def enrich_with_knowledge_graph(user_input):
    query = """
    SELECT ?relatedInfo WHERE {
        ?s ?p ?relatedInfo .
        FILTER(CONTAINS(LCASE(?relatedInfo), LCASE('meeting')))
    }
    """
    results = g.query(query)
    enriched_context = " ".join([str(row.relatedInfo) for row in results])
    return user_input + " " + enriched_context

In [ ]:
def get_contextual_embedding(user_input):
    inputs = tokenizer(user_input, return_tensors='pt')
    outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return cls_embedding

In [ ]:
def enhance_user_prompt(user_input):
    preprocessed_input = preprocess_text(user_input)
    enriched_input = enrich_with_knowledge_graph(preprocessed_input)
    embedding = get_contextual_embedding(enriched_input)
    return enriched_input, embedding

In [ ]:
user_input = "I can't go to the meeting."
enhanced_input, embedding = enhance_user_prompt(user_input)
print("Enhanced Input:", enhanced_input)
print("Embedding:", embedding)

In [29]:
pip install deep-translator

Note: you may need to restart the kernel to use updated packages.


In [32]:
import json
from deep_translator import GoogleTranslator

In [33]:
def translate_text(text, source_lang='fr', target_lang='en'):
    return GoogleTranslator(source=source_lang, target=target_lang).translate(text)

In [31]:
def translate_json_content(data, source_lang='fr', target_lang='en'):
    translated_data = {}
    for key, value in data.items():
        if isinstance(value, str):
            translated_data[key] = translate_text(value, source_lang, target_lang)
        elif isinstance(value, dict):
            translated_data[key] = translate_json_content(value, source_lang, target_lang)
        elif isinstance(value, list):
            translated_data[key] = [translate_json_content(item, source_lang, target_lang) if isinstance(item, dict) else translate_text(item, source_lang, target_lang) for item in value]
        else:
            translated_data[key] = value
    return translated_data

In [36]:
with open("formatted_data_merged.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Translate the JSON content
translated_data = translate_json_content(data, target_lang='en')

# Save the translated JSON to a new file
with open("output_english.json", "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=4)
